In [ ]:
!rm -f apache-hive-3.1.2-bin.tar.gz

In [ ]:
# Step 1: Install Java (Hive requires Java)
!apt-get update
!apt-get install -y openjdk-11-jdk wget

# Set JAVA_HOME for this session
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

# Remove previous files/folders
!rm -rf apache-hive-3.1.2-bin.tar.gz /content/hive

# Download Hive 3.1.2 archive again
!wget -q --show-progress https://archive.apache.org/dist/hive/hive-3.1.2/apache-hive-3.1.2-bin.tar.gz

# Verify size (should be ~130MB)
!ls -lh apache-hive-3.1.2-bin.tar.gz

# Extract and move
!tar -xzf apache-hive-3.1.2-bin.tar.gz
!mv apache-hive-3.1.2-bin /content/hive

# List extracted files for sanity check
!ls -l /content/hive/bin/

# Set HIVE_HOME and add bin to PATH
os.environ['HIVE_HOME'] = '/content/hive'
os.environ['PATH'] = os.environ['HIVE_HOME'] + '/bin:' + os.environ['PATH']

# Step 3: Prepare Hive warehouse directory and config
!mkdir -p /content/hive/warehouse
!mkdir -p /content/hive/tmp

# Step 4: Create a minimal hive-site.xml for local metastore (embedded Derby)
hive_site_xml = '''
<?xml version="1.0"?>
<configuration>
 <property>
  <name>javax.jdo.option.ConnectionURL</name>
  <value>jdbc:derby:;databaseName=/content/hive/metastore_db;create=true</value>
  <description>JDBC connect string for a JDBC metastore</description>
 </property>
 <property>
  <name>hive.metastore.warehouse.dir</name>
  <value>/content/hive/warehouse</value>
 </property>
 <property>
  <name>hive.exec.scratchdir</name>
  <value>/content/hive/tmp</value>
 </property>
 <property>
  <name>hive.metastore.schema.verification</name>
  <value>false</value>
 </property>
 <property>
  <name>hive.metastore.schema.verification.record.version</name>
  <value>false</value>
 </property>
</configuration>
'''
with open('/content/hive/conf/hive-site.xml', 'w') as f:
    f.write(hive_site_xml)

# Step 5: Create a Hive script with commands demonstrating databases, tables, views, functions, indexes
hive_script = '''
-- Create database
CREATE DATABASE IF NOT EXISTS testdb;
USE testdb;

-- Create table
CREATE TABLE IF NOT EXISTS employees (
  id INT,
  name STRING,
  salary FLOAT,
  dept STRING
) ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

-- Load data into table from local inline data (simulate with VALUES)
INSERT INTO employees VALUES
(1, 'Alice', 3000, 'HR'),
(2, 'Bob', 4000, 'Engineering'),
(3, 'Charlie', 3500, 'Finance'),
(4, 'David', 4500, 'Engineering');

-- Create a view on employees with salary > 3500
CREATE OR REPLACE VIEW high_salary AS
SELECT name, salary FROM employees WHERE salary > 3500;

-- Show the view data
SELECT * FROM high_salary;

-- Create a simple function: length of name (using built-in length function for demo)
-- (Custom UDFs require Java coding; we use built-in here)

-- Create index on dept column
CREATE INDEX idx_dept ON TABLE employees(dept)
AS 'COMPACT' WITH DEFERRED REBUILD;

-- Rebuild the index
ALTER INDEX idx_dept ON employees REBUILD;

-- Query employees using index
SELECT * FROM employees WHERE dept='Engineering';

'''

with open('/content/hive_script.sql', 'w') as f:
    f.write(hive_script)

# Step 6: Run the Hive script using Beeline in embedded mode (no server)
!beeline -u jdbc:hive2:// -n '' -p '' -f /content/hive_script.sql | tee /content/hive_output.log

# Step 7: Show output of script
!tail -30 /content/hive_output.log


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag